In [1]:
import psycopg2
import matplotlib.pyplot as plt

In [ ]:
plt.style.use('dark_background')

xmin = -49964.492999999784
xmax = -25724.446600000374
ymin = 155509.45260000043
ymax = 180795.91699999943

scale = 1/30000
width_in_inches = ((xmax - xmin)/0.0254)*1.1
height_in_inches = ((ymax - ymin)/0.0254)*1.1

fig = plt.figure(figsize=(width_in_inches*scale, height_in_inches*scale))
conn = psycopg2.connect("dbname=leo user=leo")
cursor_psql = conn.cursor()

######################### 
# plot only the relevant parished, i.e. the ones the STCP bus goes by
#########################

sql = """
select st_astext(st_simplify(proj_boundary,10))
from cont_aad_caop2018
join (
select st_transform(st_setsrid(st_point(stop_lon, stop_lat), 4326), 3763) as geom
from stops
) as transformed_geom on st_contains(proj_boundary, transformed_geom.geom)
where distrito = 'PORTO';
"""

cursor_psql.execute(sql)
results = cursor_psql.fetchall()

#xmin,xmax = 0,-40000
#ymin,ymax = 170000,0

for row in results:
    xs = []
    ys = []
    points_string = row[0]
    points_string = points_string[9:-2].split(",")
    for point in points_string:
        (x,y) = point.split()
        xs.append(float(x))
        ys.append(float(y))
    plt.plot(xs,ys, color='yellow',linewidth=0.2)
    #xmin = min(xmin, min(xs))
    #xmax = max(xmax, max(xs))
    #ymin = min(ymin, min(ys))
    #ymax = max(ymax, max(ys))
#print(f"xmin {xmin}, xmax {xmax}")
#print(f"ymin {ymin}, ymax {ymax}")

######################### 
# plot the bus routes
######################### 

ids = {}
id = 0
sql = "SELECT shape_id from shapes;"
cursor_psql.execute(sql)
results = cursor_psql.fetchall()

shapes = []

id = 0
for s in results:
    ids[s[0]] = id
    id += 1
    shapes.append([])

sql = """
select shape_id, 
st_astext(st_transform(st_setsrid(st_point(shape_pt_lon, shape_pt_lat), 4326), 3763)) as geometry,
shape_pt_sequence
from shapes
"""
cursor_psql.execute(sql)
results = cursor_psql.fetchall()

dic = {}
dic_id = 0

for result in results:
    id_p = result[0]
    id_s = ids[id_p]
    sequence = int(result[2])
    ps = result[1][6:-1].split()
    x = float(ps[0])
    y = float(ps[1])
    shapes[id_s].append((sequence,x,y))

for s in shapes:
    s = sorted(s,key=lambda x:s[0])

for row in shapes:
    xs = []
    ys = []
    for p in row:
        x = float(p[1])
        y = float(p[2])
        xs.append(x)
        ys.append(y)
    plt.plot(xs,ys,color='gray',alpha=0.2)

######################### 
# plot the bus stops
######################### 

sql = """
select st_astext(st_transform(st_setsrid(st_point(stop_lon,stop_lat),4326),3763))
from stops
"""
cursor_psql.execute(sql)
results = cursor_psql.fetchall()

xs = []
ys = []
for row in results:
    point_string = row[0]
    point_string = point_string[6:-1]
    x,y = point_string.split(" ")
    xs.append(float(x))
    ys.append(float(y))
plt.scatter(xs,ys,s=1,zorder=3,color='red')

'''
tentativas de querys para encontrar onde começa o serviço de taxi
mais perto de cada paragem stcp

sql = '''
select st_astext(st_setsrid(st_point(s.stop_lon, s.stop_lat), 4326)::geography) as p1,
st_astext(t.initial_point::geography) as p2
from stops s
cross join taxi_services t
limit 1
''' works

sql = """
select 
stop_id,
id,
st_distance(
    st_setsrid(st_point(s.stop_lon, s.stop_lat), 4326),
    initial_point
) as distance
from stops as s
cross join taxi_services as t
-- order by distance -- this takes minutes to run
limit 5
""" works
 
'''

Descobrir que stop é que está mais perto de cada initial_point do taxi_service

In [9]:
conn = psycopg2.connect("dbname=leo user=leo")
cursor_psql = conn.cursor()
sql = '''
select s.stop_id,
st_astext(st_transform(st_setsrid(st_point(s.stop_lon, s.stop_lat), 4326),3763)) as sp,
st_astext(st_transform(t.initial_point, 3763)) as ip
from stops s
cross join taxi_services t
order by st_distance(st_setsrid(st_point(s.stop_lon, s.stop_lat), 4326),t.initial_point)
limit 10
'''
cursor_psql.execute(sql)
results = cursor_psql.fetchall()
for result in results[:5]:
    print(result)

('HSA5', 'POINT(-41078.847939939296 164403.65921647602)', 'POINT(-41078.834564523844 164403.6491458325)')
('HSA5', 'POINT(-41078.847939939296 164403.65921647602)', 'POINT(-41078.834564523844 164403.6491458325)')
('HSA5', 'POINT(-41078.847939939296 164403.65921647602)', 'POINT(-41078.834564523844 164403.6491458325)')
('STLN1', 'POINT(-41218.244809381074 167944.8838838612)', 'POINT(-41218.271058716084 167944.84182868706)')
('COMB4', 'POINT(-39017.06409720385 166362.51917483468)', 'POINT(-39017.059393896976 166362.4569556811)')


Para cada stop_id, descobrir que autocarros é que passam por lá, a essa hora ou mais tarde.

In [13]:
conn = psycopg2.connect("dbname=leo user=leo")
cursor_psql = conn.cursor()
for stop_id, stop_point, initial_point in results:
    sql = f'''
        select trip_id, arrival_time
        from stop_times 
        where stop_id = '{stop_id}'
        order by stop_id
        limit 2
    '''
    cursor_psql.execute(sql)
    results1 = cursor_psql.fetchall()
    for result in results1:
        print(result)

('12M_0_D_2', datetime.time(1, 3))
('12M_0_D_4', datetime.time(2, 3))
('12M_0_D_2', datetime.time(1, 3))
('12M_0_D_4', datetime.time(2, 3))
('12M_0_D_2', datetime.time(1, 3))
('12M_0_D_4', datetime.time(2, 3))
('304_0_D_1', datetime.time(6, 24))
('304_1_D_2', datetime.time(6, 30))
('806_1_D_2', datetime.time(15, 24, 40))
('806_1_D_4', datetime.time(18, 24, 40))
('205_0_S_3', datetime.time(6, 53, 21))
('205_0_S_5', datetime.time(8, 40, 7))
('204_0_S_73', datetime.time(18, 6, 53))
('204_0_S_75', datetime.time(19, 56, 53))
('904_1_S_1', datetime.time(6, 18, 44))
('904_1_S_3', datetime.time(7, 18, 44))
('1M_1_U_1', datetime.time(0, 42, 57))
('1M_1_U_3', datetime.time(1, 42, 57))
('900_1_U_1', datetime.time(6, 29))
('900_0_U_2', datetime.time(6, 40))


Para cada trip_id, ver a sua trajetória para descobrir qual delas é que nos deixa mais perto do final_point

fazer uma collections de stops para esse autocarro, e fazer nearest point dentro da collection

In [18]:
conn = psycopg2.connect("dbname=leo user=leo")
cursor_psql = conn.cursor()
for trip_id, _ in results1:
    sql = f'''
        select st_astext(st_transform(t.final_point, 3763)) as fp,
        st_collect(s.stop_id)
        from stops s
        cross join taxi_services t
        order by st_closestpoint(st_setsrid(st_point(s.stop_lon, s.stop_lat), 4326),t.final_point)
    '''
cursor_psql.execute(sql)
results2 = cursor_psql.fetchall()
for result in results2[:5]:
    print(result)

UndefinedFunction: function st_collect(character varying) does not exist
LINE 3:         st_collect(s.stop_id)
                ^
HINT:  No function matches the given name and argument types. You might need to add explicit type casts.


Uma das métricas que temos também de adicionar ao data warehouse, é o número de requests de cada route_id

In [ ]:
conn = psycopg2.connect("dbname=leo user=leo")
cursor_psql = conn.cursor()
sql = '''
    select trip_id, count(*) as requests
    from ???
    group by trip_id;
'''

cursor_psql.execute(sql)
results2 = cursor_psql.fetchall()
for result in results2[:5]:
    print(result)